In [6]:
import json,itertools
import numpy
import collections
import csv 
import os
from pathlib import Path

def Remove_PC_Name(path_name):
    New_Path_Name=[]
    if 'test' in path_name:
        r_split=path_name.split("\\")
        for strings in r_split:
            if 'test' in strings:
                strings="guest"
            New_Path_Name.append(strings)
        New_Path_Name="\\".join(New_Path_Name)
        return New_Path_Name
    return path_name
   
def Remove_File_name_Reg(reg):
    if 'RASMANCS' in reg:
        r_split=reg.split("\\")
        new_reg=[]
        for m in r_split:
            if 'RASMANCS' in m:
                m='RASMANCS'
            new_reg.append(m)
        new_reg="\\".join(new_reg)
        return new_reg
    if 'RASAPI32' in reg:
        r_split=reg.split("\\")
        new_reg=[]
        for m in r_split:
            if 'RASAPI32' in m:
                m='RASAPI32'
            new_reg.append(m)
        new_reg="\\".join(new_reg)
        return new_reg
    return reg

# def WriteDictToCSV(dict_data,csv_header,header):
#     # dict_data ---  key(string): value
#     # csv_header --- the replace key --- numerical data
#     # header ---     
#     try:
#         with open('Dataset.csv', 'a') as csvfile:
#             # a means add
#             writer = csv.DictWriter(csvfile, fieldnames=csv_header)
#             if int(header)==1:
#                 writer.writeheader()
#             writer.writerow(dict_data)
#     except IOError as err:
#         errno, strerror = err.args
#         print("I/O error({0}): {1}".format(errno, strerror))    
#     return    

def WriteDictToCSV(dict_data):
    # dict_data ---  key(string): value
    # csv_header --- the replace key --- numerical data
    # header ---     
    try:
        with open('Dataset.csv', 'a') as csvfile:
            # 'a' means add
            writer = csv.DictWriter(csvfile, fieldnames=dict_data.keys())
            writer.writeheader()
            writer.writerow(dict_data)
    except IOError as err:
        errno, strerror = err.args
        print("I/O error({0}): {1}".format(errno, strerror))    
    return    

def Extract_Registry(Resource,Resources_header, filename , file_list ):
    with open(filename, 'r') as fr:
        data = json.load(fr)
    file_name=file_list
    for name in file_name:
        if name is 'apistats':
            if 'behavior' in data:
                if 'apistats' in data['behavior']:
                    for m in data['behavior']['apistats']:
                        # there are some number in json parsing therefore pid is initilize
                        pid=m
                        api_data=data['behavior']['apistats'][pid]
                    for api in api_data:
                        if str(name)+"_"+api in Resources_header.keys():
                            Resource[Resources_header["apistats_"+api]]=1
        elif name is 'strings':
            if 'strings' in data:
                for strrng in data['strings']:
                    if "strings_"+strrng in Resources_header.keys():
                         Resource[Resources_header["strings_"+strrng]]=1

        elif name is 'network':
            ipList=[]
            if 'network' in data:
                if 'udp' in data['network']:
                    for indicies in data['network']['udp']:
                        ipList.append(indicies['src'])
                        ipList.append(indicies['dst'])
                if 'hosts' in data['network']:
                    for indicies in data['network']['hosts']:
                        ipList.append(indicies)
                ipList=list(set(ipList))
                ips=[ip for ip in ipList if not (ip.startswith('192') or ip in ['224.0.0.252','8.8.8.8','239.255.255.250'])]
                for ip in ips:
                    if ip in Resources_header.keys():
                        Resource[Resources_header["network_"+str(ip)]]=1

        elif name is 'domain':
            Domain_List=[]
            if 'domains' in data['network']:
                for indices in data['network']['domains']:
                    Domain_List.append(indices['domain'])
            if 'dns' in data['network']:
                for indices in data['network']['dns']:
                    Domain_List.append(indices['request'])
            
            Domain_List=list(set(Domain_List))
            Domains=[domain for domain in Domain_List if not ('microsoft' in domain or 'msftncsi' in domain )]
            for domain in Domains:
                if domain in Resources_header.keys():
                   Resource[Resources_header["domain_"+str(domain)]]=1

        elif name is 'dropped':
            if 'metadata' in data:
                if 'output' in data['metadata']:
                    for m in data['metadata']['output']:
                        if 'dropped' in m:
                            c=0
                            for drop_name in data['metadata']['output']['dropped']:
                                if 'basename' in drop_name:
                                    for m in drop_name:
                                        basename=data['metadata']['output']['dropped'][c]['basename']
                                        if 'virusshare' not in basename:
                                            r_split=basename.rsplit('.')
                                            if "dropped_"+str(r_split[-1]) in Resources_header.keys():
                                               Resource[Resources_header["dropped_"+str(r_split[-1])]]=1

                                c+=1                   
        elif name.startswith('ext'):
            if 'behavior' in data:
                if 'summary' in data['behavior']:
                    for m in data['behavior']['summary']:
                        if str(name[3:]) in m:
                            for key in data["behavior"]['summary'][str(name[3:])]:
                                key=Remove_PC_Name(key)
                                r_split=key.rsplit('.')
                                if 'VirusShare' not in r_split[-1]:
                                    if str(name)+"_" + r_split[-1] in Resources_header.keys():
                                        Resource[Resources_header[str(name)+"_"+r_split[-1]]]=1
        else: 
            if 'behavior' in data:
                if 'summary' in data['behavior']:
                    for m in data['behavior']['summary']:
                        if str(name) in m:
                            for reg_key in data["behavior"]['summary'][str(name)]:
                                reg_key=Remove_File_name_Reg(reg_key)
                                reg_key=Remove_PC_Name(reg_key)
                                if str(name)+"_"+reg_key in Resources_header.keys():
                                    Resource[Resources_header[str(name)+"_"+reg_key]]=1
                                    print(str(name)+"_"+reg_key)
    
if __name__ == '__main__':
    
    Resource= {}
    Resources_header={}
    file_name=['regkey_opened','regkey_read','regkey_written','regkey_deleted','file_created',
               'file_opened','file_read','file_deleted','extfile_opened','extfile_read','extfile_created',
               'extfile_deleted','directory_created','dll_loaded','apistats','network','dropped','strings']
                             
    # stop =sum(1 for line in open('Dynfeatures.txt', 'rb'))
    # use the features list for ransomware
    stop =sum(1 for line in open('features_normal.txt', 'r'))

    # feature = open("features_normal.txt","r",encoding="utf-8")
    feature = open("features_normal.txt","r")    
    for ind in itertools.islice(feature , stop):
        fields = ind.split(':', 1)
        Resources_header[fields[1].strip()] = fields[0]
    feature.close()
    Resource = dict.fromkeys(Resources_header.values(),0)
    Resource[Resources_header['Label']]='label'
    Resource[Resources_header['Ransomware_Family']]="Un known"
    # define the dict with key(num): value
    dict_data = {}
    for root, filepath, filename in os.walk("E:/Analysis-on-GAN/dataset/Ransom"):
        for report in filename:
            report = Path(root).joinpath(report).as_posix()
            Extract_Registry(Resource, Resources_header, report , file_name)
            for k,v in Resources_header.items():
                dict_data[str(v)] = Resource[str(v)]
            print("     Dynamic FEATURE PARSER on Sample.json\n\n ")
            # ================ write to csv =====================
            csv_header = [i for i in range(1,len(Resources_header)+1)]
            # WriteDictToCSV(dict_data, csv_header, header)
            WriteDictToCSV(dict_data)    
# ================= calculate the 1 or 0 =================  
    # count_list = []
    
    # for k,v in Resources_header.items():
    #     # print(k,"  :  ",Resource[str(v)])
    #     dict_data[str(v)] = Resource[str(v)]
    #     # print(str(v),"  :   ", Resource[str(v)])
    # #     count_list.append(Resource[str(v)])
    # # print(collections.Counter(count_list))
    # # print(collections.Counter([Resource[str(v)]]))
    # # print(dict_data.keys())
    # # print(dict_data.values())



entVersion\explorer\FolderDescriptions\{1B3EA5DC-B587-4786-B4EF-BD1DC332AEAE}\Roamable
regkey_read_HKEY_LOCAL_MACHINE\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\explorer\FolderDescriptions\{C4AA340D-F20F-4863-AFEF-F87EF2E6BA25}\Security
regkey_read_HKEY_LOCAL_MACHINE\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\explorer\FolderDescriptions\{2400183A-6185-49FB-A2D8-4A392A602BA3}\Security
regkey_read_HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Explorer\MountPoints2\CPC\Volume\{3ffe1cfb-3eb4-11ea-9e4c-806e6f6e6963}\Data
regkey_read_HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Explorer\Advanced\Hidden
regkey_read_HKEY_LOCAL_MACHINE\SYSTEM\ControlSet001\SERVICES\LanmanWorkstation\Parameters\RpcCacheTimeout
regkey_read_HKEY_LOCAL_MACHINE\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\explorer\FolderDescriptions\{905E63B6-C1BF-494E-B29C-65B732D3D21A}\FolderTypeID
regkey_read_HKEY_LOCAL_MACHINE\SOFTWARE\Wow6432Node\Microsoft\Windows\Curr

JSONDecodeError: Expecting property name enclosed in double quotes: line 317553 column 25 (char 13834725)